In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q] 
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzles Introducción

Primero se consideran 5 preguntas que servirán como introduccion a la logica 
knight-knave para aquellos que no estén familiarizados con ella y como un
breve curso recordatorio para los que lo están.

#### 5. Suppose A instead says: "My brother and I are the same type; we are either both knights or both knaves." What could then be deduced about A and B? Suppose A had instead said: "My brother and I are different types." What can then be deduced? 

If B were a knave, no native would claim to be the same type as B, because that would be tantamount to claiming to be a knave. Therefore B must be a knight, since A did claim to be of the same type as B. As for A, he could
be either a knight or a knave.
If A had instead said that he and B were diffirent types, this would be equivalent to the statement "One of us is a knight and one of us is a knave," which in turn is the same as the statement "Exactly one of us is a knave." This is really the same as Question 4, and so the answer is that B is a knave and A is indeterminate.
Looked at another way, if B were a knight, then no inhabitant would claim to be a diffirent type than B! 

In [9]:
def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knight) = ???

// x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = ???
// El Either[x.Knave, x.Knight] es una tautologia, siempre se puede deducir un valor u otro 

defined function puzzle5_1

In [12]:
//V2 - PREMISA CORREGIDA
/*
Yo antes tenia:

x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (Either[x.Knave, x.Knight],y.Knight) = ???

Pero el Either[x.Knave, x.Knight] es una tautologia, siempre se puede deducir un valor u otro 

Por eso la premisa queda:

x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = ???
*/

def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = ???


defined function puzzle5_1

In [21]:
//V2 - PREMISA CORREGIDA

def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = 
    ({xSay: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => 
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) =>
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = 
                                        premises.P2[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2L: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_2L(xSay)
            
                (x_3L match{
                    case Left(t1: (x.Knight,y.Knight)) => t1._2 : y.Knight
                    case Right(t2: (x.Knave,y.Knave)) => t2._1(xKnight) : Nothing
                }): (y.Knight)
            
            case Right(xKnave: x.Knave) =>
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = 
                                        premises.P3[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] => Nothing = x_2R(xSay)
            
                premises.P1(y) match{
                    case Left(yKnight: y.Knight) => yKnight : (y.Knight)
                    case Right(yKnave: y.Knave) => x_3R(Right(xKnave, yKnave)) : (y.Knight)
                    // x_3R(Right(xKnave, yKnave)) devuelve un Nothing que es equivalente al tipo y.Knight
                }
            
        }) : (y.Knight)})

defined function puzzle5_1

In [20]:
//V2 - PREMISA CORREGIDA - SIMPLIFICADO

def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                val x_1L: x.Knight => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = 
                                        premises.P2[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2L: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_2L(xSay)
            
                x_3L match{
                    case Left(t1: (x.Knight,y.Knight)) => t1._2 : y.Knight
                    case Right(t2: (x.Knave,y.Knave)) => t2._1(xKnight) : Nothing
                    // Para la rama Right lo que uso es la definicion del tipo Knave:
                    // type Knave = ¬[Knight] // Knight => Nothing
                    // Por tanto si tengo un Knave que es t2._1 y le paso un Knight, que es xKnight, entonces tengo Nothing
                }
            
            case Right(xKnave) =>
                val x_1R: x.Knave => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = 
                                        premises.P3[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] => Nothing = x_2R(xSay)
            
                // En el pattern matching hay que definir ambas ramas (left y right)
                premises.P1(y) match{
                    case Left(yKnight: y.Knight) => yKnight : (y.Knight)
                    case Right(yKnave: y.Knave) => x_3R(Right(xKnave, yKnave)) : (y.Knight)
                    // x_3R(Right(xKnave, yKnave)) devuelve un Nothing que es equivalente al tipo y.Knight
                }
        }) 
    })

defined function puzzle5_1

In [18]:
// VERSION 3 YA SIMPLIFICADO

def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                val x_1L: x.Knight => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = 
                                        premises.P2[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2L: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_2L(xSay)
            
                x_3L match{
                    case Left(t1: (x.Knight,y.Knight)) => t1._2 : y.Knight
                    case Right(t2: (x.Knave,y.Knave)) => t2._1(xKnight) : Nothing
                    // Para la rama Right lo que uso es la definicion del tipo Knave:
                    // type Knave = ¬[Knight] // Knight => Nothing
                    // Por tanto si tengo un Knave que es t2._1 y le paso un Knight, que es xKnight, entonces tengo Nothing
                }
            
            case Right(xKnave) =>
                val x_1R: x.Knave => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = 
                                        premises.P3[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] => Nothing = x_2R(xSay)
            
                // En el pattern matching hay que definir ambas ramas (left y right)
                premises.P1(y) match{
                    case Left(yKnight: y.Knight) => yKnight : (y.Knight)
                    case Right(yKnave: y.Knave) => x_3R(Right(xKnave, yKnave)) : (y.Knight)
                    // x_3R(Right(xKnave, yKnave)) devuelve un Nothing que es equivalente al tipo y.Knight
                }
            
        }) 
    })

defined function puzzle5_1

In [18]:
// Sugar version

/*def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                val x_1L: x.Knight => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = 
                                        premises.P2[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2L: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_2L(xSay)
            
                x_3L match{
                    case Left(t1: (x.Knight,y.Knight)) => t1._2 : y.Knight
                    case Right(t2: (x.Knave,y.Knave)) => t2._1(xKnight) : Nothing
                }
            
            case Right(xKnave) =>
                val x_1R: x.Knave => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = 
                                        premises.P3[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2R: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] = x_1R(xKnave)
                val x_3R: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] => Nothing = x_2R(xSay)
            
                premises.P1(y) match{
                    case Left(yKnight: y.Knight) => yKnight : (y.Knight)
                    case Right(yKnave: y.Knave) => x_3R(Right(xKnave, yKnave)) : (y.Knight)
                }
        }) 
    })
*/

def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) =
        xSayxKnightyKnightOrxKnaveyKnave =>
            premises.eitherKnightOrKnave(x).fold(
                xIsKnight =>
                    val aux1 = premises.noKnightLies(x)(xIsKnight)(xSayxKnightyKnightOrxKnaveyKnave): Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]
                    aux1 match{
                        xKnightyKnight => xKnightyKnight._2
                        xKnaveyKnave => xKnaveyKnave._1(xIsKnight)
                    },
                xIsKnave =>
                    val aux2 = premises.noKnaveTellsTruth(x)(xIsKnave)(xSayxKnightyKnightOrxKnaveyKnave): ¬[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]]
                    premises.eitherKnightOrKnave(y).fold(
                        yIsKnight =>
                            yIsKnight
                        ,
                        yIsKnave =>
                            aux2(Right(xIsKnave, yIsKnave))
                    )
            )


(console):37:21 expected ")"
// Sugar version

/*def puzzle5_1(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => (y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                val x_1L: x.Knight => x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = 
                                        premises.P2[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]](x)
                val x_2L: x.Says[Either[(x.Knight,y.Knight),(x.Knave,y.Knave)]] => Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[(x.Knight,y.Knight),(x.Knave,y.Knave)] = x_2L(xSay)
            
                x_3L match{
                    case Left(t1: (x.Knight,y.Knight)) => t1._2 : y.Knight
                    case Right(t2: (x.Knave,y.Knave)) => t2._1(xKnight) : Nothing
                }
            
            case Rig

: 

In [ ]:
def puzzle5_2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[(x.Knight,y.Knave),(x.Knave,y.Knight)]] => (Either[x.Knave, x.Knight],y.Knave) = ???

// Seria el mismo caso que en el puzzle 4